In [1]:
import torch

In [5]:
model = torch.load("/content/mobilenetv3_garbage_classifier_full.pt", weights_only=False, map_location=torch.device('cpu'))
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [6]:
from torchvision.models.quantization import mobilenet_v3_large

In [7]:
model.fuse_model()

AttributeError: 'MobileNetV3' object has no attribute 'fuse_model'

In [11]:
from torchvision.models.quantization import mobilenet_v3_large
import torch.nn as nn

# Load your weights
model = torch.load("/content/mobilenetv3_garbage_classifier_full.pt", weights_only=False, map_location=torch.device('cpu'))
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [12]:
import torch.quantization

# Set quantization config
model.qconfig = torch.quantization.get_default_qconfig("qnnpack")

# Prepare model for static quantization
torch.quantization.prepare(model, inplace=True)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(
        3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
      (1): BatchNorm2d(
        16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
      (2): Hardswish(
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(
            16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False
            (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
          )
          (1): BatchNorm2d(
            16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True
            (activation_post_proc

In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import kagglehub

IMG_SIZE = 224
BATCH_SIZE = 32
data_dir = f"{kagglehub.dataset_download('siddhantmaji/unified-waste-classification-dataset')}/content/unified_dataset"

# Preprocessing (no augmentation)
calib_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Use a subset of the dataset for calibration
val_dataset = datasets.ImageFolder(root=data_dir, transform=calib_transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)


100%|██████████| 955M/955M [00:24<00:00, 41.0MB/s]

Extracting files...


In [14]:
with torch.no_grad():
    for i, (images, _) in enumerate(val_loader):
        model(images.to("cpu"))
        if i >= 20:
            break

In [15]:
torch.quantization.convert(model, inplace=True)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): QuantizedConv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), scale=0.41373685002326965, zero_point=133, padding=(1, 1), bias=False)
      (1): QuantizedBatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): QuantizedHardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.6793187856674194, zero_point=97, padding=(1, 1), groups=16, bias=False)
          (1): QuantizedBatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): QuantizedConv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), scale=0.12294790893793106, zero_point=117, bias=False)
          (1): QuantizedBatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats

In [16]:
quantized_scripted_model = torch.jit.script(model)
torch.jit.save(quantized_scripted_model, "/content/mobilenetv3_quant_jit.pt")

In [17]:
from google.colab import files
files.download("/content/mobilenetv3_quant_jit.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>